In [ ]:
#-- Install ultralytics for YOLO -------------------------------------------------------------------------
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
#-- Install GroundingDINO for Zero-Shot ---------------------------------------------------------------------
%cd /kaggle/working/  

!git clone https://github.com/IDEA-Research/GroundingDINO.git

%cd GroundingDINO/
!pip install -e .

!mkdir weights
%cd weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

%cd /kaggle/working/GroundingDINO    

#-- clear output --
from IPython import display
display.clear_output()  

!python -c "import groundingdino" && echo "Module installed successfully" || echo "Module installation failed"

In [ ]:
#-- Import -----------------------------------------------------------------------------------------------
%cd /kaggle/working/GroundingDINO
from groundingdino.util.inference import load_model as dn_load_model
from groundingdino.util.inference import load_image as dn_load_image
from groundingdino.util.inference import predict as dn_predict
from groundingdino.util.inference import annotate as dn_annotate
%cd /kaggle/working

from ultralytics import YOLO
import yaml

import torch

import pandas as pd
import numpy as np

import os
import shutil
import PIL
import random
import cv2
import matplotlib.pyplot as plt

In [ ]:
#-- Initialize -------------------------------------------------------------------
output_path = '/kaggle/working/'
input_path = '/kaggle/input/'

model_config_file = output_path + 'GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
model_weights_file = output_path + 'GroundingDINO/weights/groundingdino_swint_ogc.pth'

ds_path = input_path + 'food41/images/'

dino_imgs_path = output_path + 'dino_results/images/'
dino_lbls_path = output_path + 'dino_results/labels/'

train_dir = output_path + 'yolo_train/images/'
val_dir = output_path + 'yolo_valid/images/'
test_dir = output_path + 'yolo_test/images/'

train_lbl_dir = output_path + 'yolo_train/labels/'
val_lbl_dir = output_path + 'yolo_valid/labels/'
test_lbl_dir = output_path + 'yolo_test/labels/'

data_config_file = output_path + 'data.yaml'
test_data_config_file = output_path + 'test_data.yaml'

NUM_EPOCHS = 100

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:' , DEVICE)

PROJECT = 'model_yolo'

CONF_THRESHOLD = 0.5
IOU_THRESHOLD = 0.6

In [ ]:
#-- Get All Images from DS -----------------------------------------------------------
image_files = []
for root, dirs, files in os.walk(ds_path):
    for file in files:
        if file.endswith('.jpg'):
            image_files.append(os.path.join(root, file)) 
    
total_images = len(image_files)
print('total_images:' , total_images)  


In [ ]:
sampled_image_files = random.sample(image_files, 100)
sampled_images_to_show = random.sample(sampled_image_files, 4)

In [ ]:
def plot_boxes(image, boxes, logits, phrases):
    
    annotated_img = dn_annotate(image_source=image,
                                    boxes=boxes,
                                    logits=logits,
                                    phrases=phrases)
        
    out_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)        
    plt.imshow(out_img, interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])        
    plt.show()   
    

In [ ]:
#-- Run DINO and predict Bounding Boxes for Objects ------------------------------------------
def run_dino(model_dino, image, text_prompt='food', box_threshold=0.4, text_threshold=0.1):
    boxes, logits, phrases = dn_predict(
        model = model_dino,
        image = image,
        caption = text_prompt,
        box_threshold = box_threshold,
        text_threshold = text_threshold
    )
    return boxes, logits, phrases

In [ ]:
#-- Annotate All Images with predicted bounding boxes ------------------------------------
def annotate(dino, image_files, result_imgs_path, result_lbls_path):
    
    os.makedirs(result_imgs_path, exist_ok=True)
    os.makedirs(result_lbls_path, exist_ok=True)
    
    i = 1
    for img_file in image_files:
        
        #-- log --
        if i==1 or i%100==0:
            print(f'annotating {i}th image -------------')
        
        title = 'img_' + str(i)
        image_path = result_imgs_path + title + '.png'
        label_path = result_lbls_path + title + '.txt'
        
        img = PIL.Image.open(img_file)
        img = img.resize((640, 640))
        img.save(image_path)
        
        image_source, image = dn_load_image(image_path)
        boxes, logits, phrases = run_dino(dino, image)
        
        label = ['0 ' + ' '.join(list(map(str, b))) for b in boxes.tolist()]
        label = '\n'.join(label)
        with open(label_path, 'w') as f:
            f.write(label)
            
        if img_file in sampled_images_to_show:
            plot_boxes(image_source, boxes, logits, phrases)
        
        i += 1

In [ ]:
#-- Create a dino model --------------------------------------------------------
model_dino = dn_load_model(model_config_file,model_weights_file, device= DEVICE)

In [ ]:
#-- Annotate All Images --------------------------------------------------------
annotate(dino = model_dino ,
         image_files = sampled_image_files,
         result_imgs_path = dino_imgs_path,
         result_lbls_path = dino_lbls_path)

In [ ]:
#-- Split Dataset to Train, Test, and Validation --------------------------------------------------
def split_dataset(img_source_dir, lbl_source_dir, train_dir, test_dir, val_dir, train_lbl_dir, test_lbl_dir, val_lbl_dir, split_ratio=(0.7, 0.1, 0.2)):
    
    image_files = [file for file in os.listdir(img_source_dir) if file.endswith('.png')]
    labels_file = [file for file in os.listdir(lbl_source_dir) if file.endswith('.txt')]
    total_images = len(image_files)
    
    print('total_images:' , total_images) 
    print('labels_file:' , len(labels_file))

    # Calculate the number of files for each split
    num_train = int(split_ratio[0] * total_images)
    num_test = int(split_ratio[1] * total_images)
    num_validation = total_images - num_train - num_test
    
    print(f'train_size = {num_train}\nval_size = {num_validation}\ntest_size = {num_test}')    

    # Randomly shuffle the list of files
    random.shuffle(image_files)
    
    # Remove existing train, test, and validation directories
    for directory in [train_dir, test_dir, val_dir, train_lbl_dir, test_lbl_dir, val_lbl_dir]:
        if os.path.exists(directory):
            shutil.rmtree(directory)

    # Create train, test, and validation directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(test_lbl_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)
    
    

    # Copy files to the corresponding directories
    for i, img_file in enumerate(image_files):        
        img_title = img_file.split('.')[0]
        lbl_file = img_title + '.txt'
        
        if lbl_file in labels_file:   
            
            if i < num_train:
                dest_img_dir = train_dir
                dest_lbl_dir = train_lbl_dir
            elif i < num_train + num_test:
                dest_img_dir = test_dir
                dest_lbl_dir = test_lbl_dir
            else:
                dest_img_dir = val_dir
                dest_lbl_dir = val_lbl_dir

            # Copy file (both image and label)
            shutil.copy(os.path.join(img_source_dir, img_file), os.path.join(dest_img_dir, img_file))
            shutil.copy(os.path.join(lbl_source_dir, lbl_file), os.path.join(dest_lbl_dir, lbl_file))

In [ ]:
split_dataset(dino_imgs_path,
              dino_lbls_path,
              train_dir,
              test_dir,
              val_dir,
              train_lbl_dir,
              test_lbl_dir,
              val_lbl_dir)

In [ ]:
#-- Create Config File for YOLO --------------------------------------------------
config = {
    'names': ['food'],
    'nc': 1,
    'train': 'yolo_train/images',
    'val': 'yolo_valid/images',
    'test': 'yolo_test/images'
}

with open(data_config_file, 'w') as f:
    yaml.dump(config, f)

    
test_config = {
    'names': ['food'],
    'nc': 1,
    'train': 'yolo_train/images',
    'val': 'yolo_test/images',    
}

with open(test_data_config_file, 'w') as f:
    yaml.dump(test_config, f)

In [ ]:
#-- DS Size ------------------------------------------------------------------
train_size = len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])
val_size = len([name for name in os.listdir(val_dir) if os.path.isfile(os.path.join(val_dir, name))])
test_size = len([name for name in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, name))])

print(f'train size:{train_size}\nvalidation size:{val_size}\ntest size:{test_size}')

In [ ]:
#-- Train, EValuate, and Save Results -----------------------------------------------------
def run_yolo(model , project_name, results_file):
    
    #-- Create DF for save results --
    cols_names = ['val_or_test','conf', 'iou', 'map_50_95', 'map_50', 'map_75', 'maps']        
    df_results = pd.DataFrame(columns=cols_names)
    
    #-- Train --
    print('Training Model ---------------------------------------------------------')
    model.train(data = data_config_file,
              epochs = NUM_EPOCHS,
              device = DEVICE,
              val = True,
              save = True,
              exist_ok = True,
              plots=True,
              project = project_name,
              name = 'train')
    
    #-- load best model --
    best_model_file = output_path + project_name + '/train/weights/best.pt'
    best_model = YOLO(best_model_file) 
    
    #-- Evaluate Model on Val Data --
    print('Evaluating Model On Val Data --------------------------------------------')
    metrics = best_model.val(data = data_config_file,
                             device = DEVICE,
                             project = project_name,
                             name = 'validation')

    map_50_95 = metrics.box.map
    map_50 = metrics.box.map50
    map_75 = metrics.box.map75
    maps = metrics.box.maps #--a list contains map50-95 of each category --
    
    results = {'val_or_test': 'val',
               'conf':None,
               'iou':None,
               'map_50_95':map_50_95,
               'map_50':map_50,
               'map_75':map_75,
               'maps':maps}

    new_df = pd.DataFrame(results, index=[0])
    df_results = pd.concat([df_results, new_df], ignore_index=True)   

    print(f'map_50_95:{map_50_95}\nmap_50:{map_50}\nmap_75:{map_75}\nmaps:{maps}')

    #-- Evaluate Model Test Data --
    print('Evaluating Model On Test Data --------------------------------------------')    
    metrics = best_model.val(data = test_data_config_file,
                             conf = CONF_THRESHOLD,
                             iou = IOU_THRESHOLD,
                             device = DEVICE,
                             project = project_name,
                             name = 'test')
    
    map_50_95 = metrics.box.map
    map_50 = metrics.box.map50
    map_75 = metrics.box.map75
    maps = metrics.box.maps #--a list contains map50-95 of each category --  
    results = {'val_or_test': 'test',
               'conf':CONF_THRESHOLD,
               'iou':IOU_THRESHOLD,
               'map_50_95':map_50_95,
               'map_50':map_50,
               'map_75':map_75,
               'maps':maps}
    new_df = pd.DataFrame(results)
    df_results = pd.concat([df_results, new_df], ignore_index=True)   
    print(f'\t\tmap_50_95:{map_50_95}\nmap_50:{map_50}\nmap_75:{map_75}\nmaps:{maps}')
            
    #-- Save DF Rsults --
    df_results.to_csv(output_path + results_file, index=False)            

    
    #-- Run Best Model on Test Images and Save Results --
    print('Running Model on Test Images and Saving Results ----------------------------------')
    best_model.predict(source = test_dir,
                       conf = CONF_THRESHOLD,     
                       iou = IOU_THRESHOLD,
                       show = False,
                       save= True,
                       project= project_name +'/prediction_results',
                       name='predictions')   

In [ ]:
model = YOLO("yolov8m.pt") 
run_yolo(model =  model,
    project_name = PROJECT,
    results_file = 'yolo_results.csv')

#-- log --
print('Finished Successfully ;)')